# Dessiner un réseau de neurones automatiquement
Cette fonction génère le code $\LaTeX$ nécessaire au tracé d'un réseau de neurones.

<img width="500px" src="NN.png"/>

In [1]:
def get_legende(nb_hidden_layers):
    code =  r"""
\node[annot, above of=A-0, node distance=1cm] (hl1) {Hidden layer 1};
\node[annot, left of=hl1] {Feature layer};
"""
    for i in range(1, nb_hidden_layers):
        code += r"\node[annot, right of=hl{0}] (hl{1}) {{Hidden layer {1}}};".format(i, i+1) + '\n'
    code += r"\node[annot, right of=hl{}] {{Output layer}};".format(nb_hidden_layers) 
    code += "\n\end{tikzpicture}\n\end{document}"
    return code

In [2]:
def get_hidden_layer(hidden_layers, y_shift):
    code = r"""\foreach \name / \y in {{{0},...,{1}}}
        \path[yshift={2}cm]
            node[hidden neuron] (A-\name) at (\layersep,-\y cm) {{}};
""".format(0, hidden_layers[0]-1, y_shift)
    
    lettre = ['B','C','D','E']
    for i in range(1, len(hidden_layers)):
        code += r"""
\foreach \name / \y in {{{0},...,{1}}}
        \path[yshift={2}cm]
            node[hidden neuron] ({3}-\name) at ({4}*\layersep,-\y cm) {{}};
        """.format(0, hidden_layers[0]-1, y_shift, lettre[i-1], i+1) + '\n'
        
    return code

In [3]:
def get_output(hidden_layers, output_node, debut_feature, fin_feature):
    lettre = ['A','B','C','D','E']   
    code = r"""\node[output neuron, pin={{[pin edge={{->}}]right:Output}}] (O) at ({1}, -{0}) {{}};
""".format(output_node, '\layersep+'*(len(hidden_layers))+'\layersep')
    
    code += r"""
% Connect every node in the input layer with every node in the hidden layer.
    \foreach \source in {{{0},...,{1}}}
    \foreach \dest in {{0,...,{2}}}
        \path (I-\source) edge (A-\dest);
    """.format(debut_feature, fin_feature, hidden_layers[0]-1 )
    
    for i in range(1, len(hidden_layers)):
        code += r"""
% Connection de la hidden layer {0} à la hidden layer {1} :
        \foreach \source in {{{4},...,{5}}}
        \foreach \dest in {{{4},...,{5}}}
            \path ({2}-\source) edge ({3}-\dest);
        """.format(i, i+1, lettre[i-1], lettre[i], 0, hidden_layers[0]-1 )
    
    code += r"""
% Connect every node in the hidden layer with the output layer
    \foreach \source in {{0,...,{}}}
        \path ({}-\source) edge (O);""".format(hidden_layers[0]-1, lettre[len(hidden_layers)-1])
    return code

In [4]:
def get_yshift_outputmode(nb_hidden_units, nb_features, hidden_layers):
    debut_feature = int((hidden_layers[0] - nb_features ) / 2)
    fin_feature = debut_feature + nb_features - 1
    output_node = (debut_feature+fin_feature) / 2
    y_shift = 0
    if (nb_hidden_units%2 + nb_features%2 == 1) :
        y_shift = 0.5
    return y_shift, output_node

In [5]:
def trace_neural_network(nb_features, hidden_layers):
    """
    hidden_layers : une liste où hidden_layers[i] est le nombre de hidden units dans la layer i
    """
    
    debut_feature = int((hidden_layers[0] - nb_features ) / 2)
    fin_feature = debut_feature + nb_features - 1
    
    y_shift, output_node = get_yshift_outputmode(hidden_layers[0], nb_features, hidden_layers)
    nb_hidden_layers=len(hidden_layers)
    code_LaTeX = r"""
\documentclass{{article}}

\usepackage{{tikz}}
\begin{{document}}
\pagestyle{{empty}}

\def\layersep{{2.5cm}}

\begin{{tikzpicture}}[shorten >=1pt,->,draw=black!50, node distance=\layersep]
    \tikzstyle{{every pin edge}}=[<-,shorten <=1pt]
    \tikzstyle{{neuron}}=[circle,fill=black!25,minimum size=17pt,inner sep=0pt]
    \tikzstyle{{input neuron}}=[neuron, fill=blue!50];
    \tikzstyle{{output neuron}}=[neuron, fill=purple!50];
    \tikzstyle{{hidden neuron}}=[neuron, fill=pink];
    \tikzstyle{{annot}} = [text width=4em, text centered]

% Draw the input layer nodes
\foreach \name / \y in {{{0},...,{1}}}
% This is the same as writing \foreach \name / \y in {{1/1,2/2,3/3,4/4}}
    \node[input neuron, pin=left:Feature \#\y] (I-\name) at (0,-\y) {{}};

% Draw the hidden layer nodes
{2}

% Draw the output layer node
{3}

% Annotate the layers
{4}
    """.format(debut_feature,
               fin_feature, 
               get_hidden_layer(hidden_layers, y_shift),
               get_output(hidden_layers, output_node, debut_feature, fin_feature), get_legende(len(hidden_layers))) 
    return print(code_LaTeX)

In [6]:
trace_neural_network(nb_features=4, hidden_layers=[6,6])


\documentclass{article}

\usepackage{tikz}
\begin{document}
\pagestyle{empty}

\def\layersep{2.5cm}

\begin{tikzpicture}[shorten >=1pt,->,draw=black!50, node distance=\layersep]
    \tikzstyle{every pin edge}=[<-,shorten <=1pt]
    \tikzstyle{neuron}=[circle,fill=black!25,minimum size=17pt,inner sep=0pt]
    \tikzstyle{input neuron}=[neuron, fill=blue!50];
    \tikzstyle{output neuron}=[neuron, fill=purple!50];
    \tikzstyle{hidden neuron}=[neuron, fill=pink];
    \tikzstyle{annot} = [text width=4em, text centered]

% Draw the input layer nodes
\foreach \name / \y in {1,...,4}
% This is the same as writing \foreach \name / \y in {1/1,2/2,3/3,4/4}
    \node[input neuron, pin=left:Feature \#\y] (I-\name) at (0,-\y) {};

% Draw the hidden layer nodes
\foreach \name / \y in {0,...,5}
        \path[yshift=0cm]
            node[hidden neuron] (A-\name) at (\layersep,-\y cm) {};

\foreach \name / \y in {0,...,5}
        \path[yshift=0cm]
            node[hidden neuron] (B-\name) at (2*\laye